### Name:
> Castillo, Marvien Angel C. <br>
> Herrera, Mikaela Gabrielle B. <br>
> Regindin, Sean Adrien I. <br>

# Setup Environment

In [42]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin:/usr/local/cuda/bin:/usr/local/cuda/bin


# Check if CUDA is present

In [43]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [44]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov  6 12:27:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   31C    P0             43W /  250W |    1464MiB /  32768MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Variant 1 - C Program

In [45]:
%%writefile C_var1.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
// ***C function version
void kernel_C(float A[], float B[], float C[], size_t n, int idx[]) {
	for (int i = 0; i < n; i++) {
		if (A[i] >= B[i]) {
			C[i] = A[i];
			idx[i] = 0;
		}
		else {
			C[i] = B[i];
			idx[i] = 1;
		}
	}
}

int main(int argc, char** argv)
{
   const size_t ARRAY_SIZE = 1<<28;
   const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
   const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
//number of times the program is to be executed
   const size_t loope = 30;
//declare array
   float *C,*A,*B;
   int *idx,a;
   A = (float*)malloc(FLOAT_ARRAY_BYTES);
   B = (float*)malloc(FLOAT_ARRAY_BYTES);
   C = (float*)malloc(FLOAT_ARRAY_BYTES);
   idx = (int*)malloc(INT_ARRAY_BYTES);
   a=2;
//timer variables
  clock_t start, end;
// ***--- initialize your array here ---------
   int i;
	for (i = 0; i < ARRAY_SIZE; i++) {
		A[i] = sin(i * 0.0005) * 100.0 + 50.0;
		B[i] = cos(i * 0.0003) * 100.0 + 50.0;
	}
// fill-in cache
    kernel_C(A,B,C,ARRAY_SIZE,idx);
//time here
  double elapse, time_taken;
  elapse = 0.0f;
  for (int i=0; i<loope; i++){
    start = clock();
      kernel_C(A,B,C,ARRAY_SIZE,idx );
    end = clock();
    time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
    elapse = elapse + time_taken;
  }
  printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse/loope, ARRAY_SIZE);

// error checking routine here --
   size_t err_count = 0;
   int sanity_checker = 0;
   for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Variant 1 (C) outputs match expected outputs. Errors found = %d.",sanity_checker);
  
  // Free memory
  free(A);
  free(B);
  free(C);
  free(idx);
  return 0;
}

Writing C_var1.c


In [46]:
%%bash
gcc C_var1.c -o C_var1 -lm

In [47]:
%%bash
./C_var1

Function (in C) average time for 30 loops is 4648.745000 milliseconds to execute an array size 268435456 
Variant 1 (C) outputs match expected outputs. Errors found = 0.

# Variant 2 - Grid Stride Loop

In [48]:
%%writefile CUDA_var2.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
//Grid stride loop

//*** CUDA kernel
__global__
void kernel(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<28;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 30;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
// *** init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }

// *** setup CUDA kernel
  size_t numThreads = 1024;
  size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** VARIANT 2 ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks,numThreads);
  for (size_t i=0; i<loope;i++)
    kernel <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();
  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Variant 2 outputs match expected outputs. Errors found = %d.",sanity_checker);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}


Writing CUDA_var2.cu


In [49]:
%%bash
nvcc CUDA_var2.cu -o CUDA_var2 -Wno-deprecated-gpu-targets

In [50]:
%%bash
nvprof ./CUDA_var2

==1039328== NVPROF is profiling process 1039328, command: ./CUDA_var2


*** VARIANT 2 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 2 outputs match expected outputs. Errors found = 0.

==1039328== Profiling application: ./CUDA_var2
==1039328== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  8.35349s        30  278.45ms  11.008ms  7.94593s  kernel(unsigned long, float*, float*, float*, int*)
      API calls:   77.15%  8.35394s         1  8.35394s  8.35394s  8.35394s  cudaDeviceSynchronize
                   18.67%  2.02125s         4  505.31ms  106.81us  2.01942s  cudaMallocManaged
                    4.13%  446.84ms         4  111.71ms  100.13ms  126.71ms  cudaFree
                    0.04%  4.7272ms        30  157.57us  7.4860us  3.9101ms  cudaLaunchKernel
                    0.01%  565.61us       114  4.9610us     100ns  235.50us  cuDeviceGetAttribute
                    0.00%  203.70us         1  203.70us  203.70us  203.70us  cuDeviceGetName
                    0.00%  25.400us         1  25.400us  25.400us  25.400us  cuDeviceTotalMem
                    0.00%  19.226us         3  6.4080

In [51]:
%%bash
nsys profile  -o CUDA_var2 ./CUDA_var2

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



*** VARIANT 2 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 2 outputs match expected outputs. Errors found = 0.Collecting data...
Generating '/tmp/nsys-report-7a84.qdstrm'
[1/1] [========================100%] CUDA_var2.nsys-rep
Generated:
	/home/jupyter-mikaela_herrera@dl-90f38/CUDA MP/CUDA_var2.nsys-rep


# Variant 3.0 - Grid Stride Loop with Prefetch

In [52]:
%%writefile CUDA_var3.cu
// prefetch
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
//CUDA kernel
__global__
void kernel(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<28;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 30;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** VARIANT 3 ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize(); 

//"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Variant 3 outputs match expected outputs. Errors found = %d.",sanity_checker);
//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Writing CUDA_var3.cu


In [53]:
%%bash
nvcc CUDA_var3.cu -o CUDA_var3 -Wno-deprecated-gpu-targets

In [54]:
%%bash
nvprof ./CUDA_var3

==1039804== NVPROF is profiling process 1039804, command: ./CUDA_var3


*** VARIANT 3 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 3 outputs match expected outputs. Errors found = 0.

==1039804== Profiling application: ./CUDA_var3
==1039804== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  426.11ms        30  14.204ms  11.119ms  17.153ms  kernel(unsigned long, float*, float*, float*, int*)
      API calls:   55.88%  3.67313s         8  459.14ms  15.140ms  1.07166s  cudaMemPrefetchAsync
                   32.53%  2.13867s         4  534.67ms  88.106us  2.13714s  cudaMallocManaged
                    6.50%  427.14ms         1  427.14ms  427.14ms  427.14ms  cudaDeviceSynchronize
                    5.02%  330.06ms         4  82.516ms  73.544ms  94.856ms  cudaFree
                    0.05%  3.2286ms        30  107.62us  11.453us  2.5755ms  cudaLaunchKernel
                    0.01%  658.47us       114  5.7760us     186ns  313.80us  cuDeviceGetAttribute
                    0.01%  468.65us         1  468.65us  468.65us  468.65us  cuDeviceGetName
                    0.00%  46.639us         1  46

In [55]:
%%bash
nsys profile  -o CUDA_var3 ./CUDA_var3

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



*** VARIANT 3 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 3 outputs match expected outputs. Errors found = 0.Collecting data...
Generating '/tmp/nsys-report-976f.qdstrm'
[1/1] [========================100%] CUDA_var3.nsys-rep
Generated:
	/home/jupyter-mikaela_herrera@dl-90f38/CUDA MP/CUDA_var3.nsys-rep


# Variant 4.0 - Grid Stride Loop with Prefetch and Page Creation

In [56]:
%%writefile CUDA_var4.cu
//prefetch + page creation
// page creation responsible gpu fault page
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
//CUDA kernel
__global__
void kernel(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}


int main(){
  const size_t ARRAY_SIZE = 1<<28;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 30;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);
//get gpu id
  int device = -1;
  cudaGetDevice(&device);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }
//"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;   
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** VARIANT 4 ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);

  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Variant 4 outputs match expected outputs. Errors found = %d.",sanity_checker);

//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;
}

Writing CUDA_var4.cu


In [57]:
%%bash
nvcc CUDA_var4.cu -o CUDA_var4 -Wno-deprecated-gpu-targets

In [58]:
%%bash
nvprof ./CUDA_var4

==1040071== NVPROF is profiling process 1040071, command: ./CUDA_var4


*** VARIANT 4 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 4 outputs match expected outputs. Errors found = 0.

==1040071== Profiling application: ./CUDA_var4
==1040071== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  431.24ms        30  14.375ms  11.385ms  18.089ms  kernel(unsigned long, float*, float*, float*, int*)
      API calls:   63.52%  5.41810s        12  451.51ms  38.825us  1.17772s  cudaMemPrefetchAsync
                   23.29%  1.98677s         4  496.69ms  57.861us  1.98567s  cudaMallocManaged
                    5.06%  432.01ms         1  432.01ms  432.01ms  432.01ms  cudaDeviceSynchronize
                    4.50%  384.07ms         4  96.017ms  71.549ms  135.56ms  cudaFree
                    3.60%  306.87ms        30  10.229ms  11.117us  306.00ms  cudaLaunchKernel
                    0.01%  1.2101ms       114  10.614us     169ns  469.79us  cuDeviceGetAttribute
                    0.00%  318.67us         1  318.67us  318.67us  318.67us  cuDeviceGetName
                    0.00%  132.31us         1  13

In [59]:
%%bash
nsys profile  -o CUDA_var4 ./CUDA_var4

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



*** VARIANT 4 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 4 outputs match expected outputs. Errors found = 0.Collecting data...
Generating '/tmp/nsys-report-f699.qdstrm'
[1/1] [========================100%] CUDA_var4.nsys-rep
Generated:
	/home/jupyter-mikaela_herrera@dl-90f38/CUDA MP/CUDA_var4.nsys-rep


# Variant 5.0 - Grid Stride Loop with Prefetch and Page Creation + mem advise

In [60]:
%%writefile CUDA_var5.cu
//prefetch + page creation + memadvise

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

//CUDA kernel
__global__
void kernel(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<28;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 30;
//declare array
  float *A,*B,*C; 
  int *idx;
  cudaMallocManaged(&A, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&B, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&C, FLOAT_ARRAY_BYTES);
  cudaMallocManaged(&idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// memory advise
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(A, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
   cudaMemAdvise(B, FLOAT_ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

//"prefetch data" to create CPU page memory
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
//"prefetch data" to create GPU page memory
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// ****init array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    A[i] = sin(i * 0.0005) * 100.0 + 50.0;
    B[i] = cos(i * 0.0003) * 100.0 + 50.0;
  }


 //"Prefetch data" from CPU-GPU
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,device,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,device,NULL);

// setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (ARRAY_SIZE + numThreads-1) / numThreads;

  printf("*** VARIANT 5 ***\n");
  printf("numElements = %lu\n", ARRAY_SIZE);
  printf("numBlocks = %lu, numThreads = %lu \n",numBlocks, numThreads);
  for (size_t i=0; i<loope;i++)
    kernel <<<numBlocks, numThreads>>> (ARRAY_SIZE,A,B,C,idx);
//barrier
    cudaDeviceSynchronize();

  //"Prefetch data" from GPU-CPU
  cudaMemPrefetchAsync(C,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(A,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(B,FLOAT_ARRAY_BYTES,cudaCpuDeviceId,NULL);
  cudaMemPrefetchAsync(idx,INT_ARRAY_BYTES,cudaCpuDeviceId,NULL);


  int sanity_checker = 0;
  for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (A[i] >= B[i]) ? A[i] : B[i];
        int expected_idx = (A[i] >= B[i]) ? 0 : 1;
    
        if (fabs(C[i] - expected_C) > 1e-5 || idx[i] != expected_idx) {
            sanity_checker++;
        }
    }
  printf("Variant 5 outputs match expected outputs. Errors found = %d.",sanity_checker);

//free memory
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);
  cudaFree(idx);
  return 0;

}


Writing CUDA_var5.cu


In [61]:
%%bash
nvcc CUDA_var5.cu -o CUDA_var5 -Wno-deprecated-gpu-targets

In [62]:
%%bash
nvprof ./CUDA_var5

==1040319== NVPROF is profiling process 1040319, command: ./CUDA_var5


*** VARIANT 5 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 5 outputs match expected outputs. Errors found = 0.

==1040319== Profiling application: ./CUDA_var5
==1040319== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  415.38ms        30  13.846ms  11.007ms  16.976ms  kernel(unsigned long, float*, float*, float*, int*)
      API calls:   64.27%  5.03771s        12  419.81ms  5.8213ms  1.36689s  cudaMemPrefetchAsync
                   25.90%  2.03017s         4  507.54ms  68.978us  2.02897s  cudaMallocManaged
                    5.34%  418.92ms         1  418.92ms  418.92ms  418.92ms  cudaDeviceSynchronize
                    4.44%  347.82ms         4  86.956ms  72.836ms  97.685ms  cudaFree
                    0.04%  2.9671ms        30  98.902us  7.1010us  2.4522ms  cudaLaunchKernel
                    0.01%  516.38us       114  4.5290us     121ns  191.55us  cuDeviceGetAttribute
                    0.00%  320.51us         4  80.126us  7.9380us  284.72us  cudaMemAdvise
                    0.00%  178.79us         1  178.

In [63]:
%%bash
nsys profile  -o CUDA_var5 ./CUDA_var5

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



*** VARIANT 5 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 5 outputs match expected outputs. Errors found = 0.Collecting data...
Generating '/tmp/nsys-report-54c5.qdstrm'
[1/1] [========================100%] CUDA_var5.nsys-rep
Generated:
	/home/jupyter-mikaela_herrera@dl-90f38/CUDA MP/CUDA_var5.nsys-rep


# Variant 6.0 - CUDA classic MEMCPY

In [64]:
%%writefile CUDA_var6.cu
//grid stride loop + memcpy

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

//CUDA kernel
__global__
void kernel(size_t n, float A[],float B[],float C[],int idx[]){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride)
    {
        if (A[i] >= B[i]) {
            C[i] = A[i];
            idx[i] = 0;
        }
        else {
            C[i] = B[i];
            idx[i] = 1;
        }
    }
}

int main(){
  const size_t ARRAY_SIZE = 1<<28;
  const size_t INT_ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
  const size_t FLOAT_ARRAY_BYTES = ARRAY_SIZE * sizeof(float);
 //number of times the program is to be executed
   const size_t loope = 30;

float *h_A = (float*)malloc(FLOAT_ARRAY_BYTES);
float *h_B = (float*)malloc(FLOAT_ARRAY_BYTES);
float *h_C = (float*)malloc(FLOAT_ARRAY_BYTES);
int *h_idx = (int*)malloc(INT_ARRAY_BYTES);

// ****init host array
  int i;
  for (i = 0; i < ARRAY_SIZE; i++) {
    h_A[i] = sinf(i * 0.0005) * 100.0 + 50.0;
    h_B[i] = cosf(i * 0.0003) * 100.0 + 50.0;
  }

// device allocations
  float *d_A,*d_B,*d_C;
  int *d_idx;
  cudaMalloc(&d_A, FLOAT_ARRAY_BYTES); //we dont use cudaMallocManaged with cudaMemcpy
  cudaMalloc(&d_B, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_C, FLOAT_ARRAY_BYTES);
  cudaMalloc(&d_idx, INT_ARRAY_BYTES);

//get gpu id
  int device = -1;
  cudaGetDevice(&device);

// Copy data to device
    cudaMemcpy(d_A, h_A, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, FLOAT_ARRAY_BYTES, cudaMemcpyHostToDevice);

// Kernel launch configuration
    size_t threadsPerBlock = 1024;
    size_t blocksPerGrid = (ARRAY_SIZE + threadsPerBlock - 1) / threadsPerBlock;
    printf("*** VARIANT 6 ***\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n",blocksPerGrid, threadsPerBlock);
    
  for (size_t i=0; i<loope;i++){
    kernel <<<blocksPerGrid, threadsPerBlock>>> (ARRAY_SIZE, d_A, d_B, d_C, d_idx);
    cudaDeviceSynchronize();
  }

    
// Copy results back to host
    cudaMemcpy(h_C, d_C, FLOAT_ARRAY_BYTES, cudaMemcpyDeviceToHost);
    cudaMemcpy(h_idx, d_idx, INT_ARRAY_BYTES, cudaMemcpyDeviceToHost);

    // Verification loop
    int errors = 0;
    for (int i = 0; i < ARRAY_SIZE; i++) { 
        float expected_C = (h_A[i] >= h_B[i]) ? h_A[i] : h_B[i];
        int expected_idx = (h_A[i] >= h_B[i]) ? 0 : 1;
    
        if (fabs(h_C[i] - expected_C) > 1e-5 || h_idx[i] != expected_idx) {
            errors++;
        }
    }
    printf("Variant 6 outputs match expected outputs. Errors found = %d.", errors);

// Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaFree(d_idx);

// Free host memory
    free(h_A);
    free(h_B);
    free(h_C);
    free(h_idx);

  return 0;
}

Writing CUDA_var6.cu


In [65]:
%%bash
nvcc CUDA_var6.cu -o CUDA_var6 -Wno-deprecated-gpu-targets

In [66]:
%%bash
nvprof ./CUDA_var6

==1040815== NVPROF is profiling process 1040815, command: ./CUDA_var6


*** VARIANT 6 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 6 outputs match expected outputs. Errors found = 0.

==1040815== Profiling application: ./CUDA_var6
==1040815== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   80.06%  12.8753s         2  6.43766s  6.40197s  6.47335s  [CUDA memcpy DtoH]
                   17.84%  2.86827s         2  1.43413s  1.43329s  1.43498s  [CUDA memcpy HtoD]
                    2.10%  337.72ms        30  11.257ms  11.112ms  11.381ms  kernel(unsigned long, float*, float*, float*, int*)
      API calls:   87.67%  15.7722s         4  3.94306s  1.43505s  6.48485s  cudaMemcpy
                   10.02%  1.80179s         4  450.45ms  2.1055ms  1.79446s  cudaMalloc
                    2.16%  388.29ms        30  12.943ms  12.608ms  13.281ms  cudaDeviceSynchronize
                    0.10%  18.005ms        30  600.16us  244.91us  3.2176ms  cudaLaunchKernel
                    0.05%  8.4320ms         4  2.1080ms  1.7182ms  3.0760ms  cudaFree
                    0.01%  1.4457ms       114  12.681us     136ns

In [67]:
%%bash
nsys profile  -o CUDA_var6 ./CUDA_var6

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



*** VARIANT 6 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Variant 6 outputs match expected outputs. Errors found = 0.Collecting data...
Generating '/tmp/nsys-report-7932.qdstrm'
[1/1] [========================100%] CUDA_var6.nsys-rep
Generated:
	/home/jupyter-mikaela_herrera@dl-90f38/CUDA MP/CUDA_var6.nsys-rep


# Variant 7.0 - CUDA init

In [68]:
%%writefile CUDA_var7.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// CUDA kernel to initialize A and B
__global__
void init_arrays(size_t n, float *A, float *B) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < n; i += stride) {
        A[i] = sinf(i * 0.0005f) * 100.0f + 50.0f;
        B[i] = cosf(i * 0.0003f) * 100.0f + 50.0f;
    }
}

int main() {
    const size_t ARRAY_SIZE = 1 << 28;
    const size_t FLOAT_BYTES = ARRAY_SIZE * sizeof(float);
    //number of times the program is to be executed
    const size_t loope = 30;

    // Unified memory allocation
    float *A, *B;
    cudaMallocManaged(&A, FLOAT_BYTES);
    cudaMallocManaged(&B, FLOAT_BYTES);


    // Launch kernel
    size_t threads = 1024;
    size_t blocks = (ARRAY_SIZE + threads - 1) / threads;
    printf("*** VARIANT 7 ***\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n",blocks, threads);

    for (size_t i=0; i<loope;i++){
        init_arrays<<<blocks, threads>>>(ARRAY_SIZE, A, B);
        cudaDeviceSynchronize();
    }
    
    // Verification loop (no memcpy needed)
    int error = 0;
    for (int i = 0; i < 100; i++) {
        float expected_A = sinf(i * 0.0005f) * 100.0f + 50.0f;
        float expected_B = cosf(i * 0.0003f) * 100.0f + 50.0f;
        if (fabs(A[i] - expected_A) > 1e-4f || fabs(B[i] - expected_B) > 1e-4f) {
            error++;
        }
    }
    printf("Errors found = %d\n", error);
    
    // Free unified memory
    cudaFree(A);
    cudaFree(B);

    return 0;
}

Writing CUDA_var7.cu


In [69]:
%%bash
nvcc CUDA_var7.cu -o CUDA_var7 -Wno-deprecated-gpu-targets

In [70]:
%%bash
nvprof ./CUDA_var7

==1041155== NVPROF is profiling process 1041155, command: ./CUDA_var7


*** VARIANT 7 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Errors found = 0


==1041155== Profiling application: ./CUDA_var7
==1041155== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.62274s        30  120.76ms  9.7476ms  3.33365s  init_arrays(unsigned long, float*, float*)
      API calls:   66.01%  3.67450s        30  122.48ms  11.199ms  3.33525s  cudaDeviceSynchronize
                   32.83%  1.82750s         2  913.75ms  1.3232ms  1.82618s  cudaMallocManaged
                    0.80%  44.468ms         2  22.234ms  20.435ms  24.032ms  cudaFree
                    0.34%  18.975ms        30  632.50us  223.77us  5.4305ms  cudaLaunchKernel
                    0.02%  885.08us       114  7.7630us     100ns  469.21us  cuDeviceGetAttribute
                    0.00%  254.20us         1  254.20us  254.20us  254.20us  cuDeviceGetName
                    0.00%  50.951us         1  50.951us  50.951us  50.951us  cuDeviceTotalMem
                    0.00%  29.673us         1  29.673us  29.67

In [71]:
%%bash
nsys profile  -o CUDA_var7 ./CUDA_var7

         This may increase runtime overhead and the likelihood of false
         dependencies across CUDA Streams. If you wish to avoid this, please
         disable the feature with --cuda-event-trace=false.
Try the 'nsys status --environment' command to learn more.

Try the 'nsys status --environment' command to learn more.



*** VARIANT 7 ***
numElements = 268435456
numBlocks = 262144, numThreads = 1024 
Errors found = 0
Generating '/tmp/nsys-report-d023.qdstrm'
[1/1] [========================100%] CUDA_var7.nsys-rep
Generated:
	/home/jupyter-mikaela_herrera@dl-90f38/CUDA MP/CUDA_var7.nsys-rep


---